1. MLA(Multi-Label Attention): 选 MQA 模式，让所有查询头共享同一组 KV 向量，减少计算量（核心：共享 KV，适配 DSA 的高效需求）
2. DSA(Deep Seek Attention): 拆成两步（核心：少算无用 token，降复杂度）
    - 闪电索引器(Lightning Indexer): 用简单线性层快速给“查询-历史 Token" 打分；
    - Top-k 筛选: 只保留高分的 2048 个 Token 做注意力计算
3. 结合：MLA 提供 KV 共享的基础框架，DSA 在 MLA 之上做“稀疏筛选“，最终注意力计算只针对 Top-k 的 KV 对

**实现 MLA**  
MLA 的 MQA 模式核心是 “单组 KV 供所有查询头使用”，避免重复计算 KV，适配 DSA 的稀疏逻辑。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLA_MQA(nn.Module):
    def __init__(self, d_model=512, num_query_heads=8, d_k=64):
        super().__init__()
        self.d_model = d_model
        self.num_query_heads = num_query_heads
        self.d_k = d_k

        # 1. 查询投影（多查询头）：将输入h_t投影成 num_query_heads个查询
        self.W_q = nn.Linear(d_model, num_query_heads * d_k)
        # 2. KV 投影（单组， MQA 模式）：将所有查询头共享这组 KV
        self.W_k = nn.Linear(d_model, d_k)
        self.W_v = nn.Linear(d_model, d_k)
        # 3. 输出投影
        self.W_o = nn.Linear(num_query_heads * d_k, d_model)
    
    def forward(self, x):
        # x: [batch_size, seq_len, d_model] 输入序列
        batch_size, seq_len, _ = x.shape

        # 生成查询 Q：[batch_size, num_query_heads, seq_len, d_k]
        Q = self.W_q(x).view(batch_size, seq_len, self.num_query_heads, self.d_k).transpose(1, 2)
        # 生成共享 KV：[batch_size, seq_len, d_k]
        K = self.W_k(x)  # [batch_size, seq_len, d_k]
        V = self.W_v(x)  # [batch_size, seq_len, d_k]

        return Q, K, V  # 输出Q（多头）、共享 KV

**实现 DSA**  
基于 MLA 的 KV，用“闪电索引器打分 + Top-k 筛选“实现稀疏计算，核心是“只关注高分 Token“

In [2]:
class DSA(nn.Module):
    def __init__(self, d_model=512, d_k=64, top_k=2048):
        super().__init__()
        self.top_k = top_k  # 每个查询保留 Top-k Token（原文默认：2048）
        self.d_k = d_k

        # 闪电索引器（简化版）： 快速计算查询与历史 Token 的相关性得分
        self.indexer_q = nn.Linear(d_model, d_k)  # 查询投影到索引器维度
        self.indexer_k = nn.Linear(d_model, d_k)  # 历史 Token 投影到索引器维度
        self.relu = nn.ReLU()  # 原文使用 ReLU 提升吞吐量

    def forward(self, x, Q, K, V):
        # x: [batch_size, seq_len, d_model] 原始输入（用于索引器打分）
        # Q: [batch_size, num_query_heads, seq_len, d_k] （MLA 输出的查询）
        # K，V: [batch_size, seq_len, d_k] （MLA 输出的共享 KV）
        batch_size, seq_len, _ = x.shape
        num_query_heads = Q.shape[1]

        # 1. 闪电索引器：计算每个查询 Token 与所有历史 Token 的得分I_{t,s}
        q_index = self.indexer_q(x)  # [batch_size, seq_len, d_k]
        k_index = self.indexer_k(x)  # [batch_size, seq_len, d_k]
        # 得分计算 q_index @ k_index.T --> [batch_size, seq_len, seq_len]（每个位置对所有位置的得分）
        index_scores = torch.bmm(q_index, k_index.transpose(1, 2))  # I_{t,s}
        index_scores = self.relu(index_scores)  # 激活函数

        # 2. Top-k 筛选： 每个查询 token 只保留得分最高的 top-k 个历史 token
        # top_k_values: 得分值，top_k_indices: 得分对应的位置索引
        top_k_values, top_k_indices = torch.topk(index_scores, k=min(self.top_k, seq_len), dim=-1)
        # top_k_indices: [batch_size, seq_len, top_k] （每个查询 token 对应的 top-k 个历史 token 索引）

        # 3. 提取 Top-k 对应的 KV 向量（共享 KV，所以直接按索引取）
        # 先把索引展平，方便批量提取
        batch_idx = torch.arange(batch_size).unsqueeze(-1).unsqueeze(-1).repeat(1, seq_len, self.top_k)
        # K_topk: [batch_size, seq_len, top_k, d_k]
        K_topk = K[batch_idx, top_k_indices]
        V_topk = V[batch_idx, top_k_indices]

        # 4. 稀疏注意力计算
        # 调整 Q 维度: [batch_size, num_query_heads, seq_len, d_k] --> [batch_size, seq_len, num_query_heads, d_k]
        Q_reshaped = Q.transpose(1, 2)
        # 调整K—topk 维度：[batch_size, seq_len, top_k, d_k] --> [batch_size, seq_len, d_k, top_k]
        K_topk_T = K_topk.transpose(-1, -2)
        # 注意力得分 Q @ K.T / (sqrt(d_k) -> [batch_size, seq_len, num_query_heads, top_k]
        attn_scores = torch.matmul(Q_reshaped, K_topk_T) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))
        # softmax 归一化
        attn_weights = F.softmax(attn_scores, dim=-1)
        # 注意力输出 权重 @ V --> [batch_size, seq_len, num_query_heads, d_k]
        attn_output = torch.matmul(attn_weights, V_topk)

        # 5. 重组输出：[batch_size, seq_len, num_query_heads * d_k] --> [batch_size, seq_len, d_model]
        attn_output = attn_output.reshape(batch_size, seq_len, num_query_heads * self.d_k)
        return attn_output

**结合 DSA+MLA**  
MLA 提供共享 KV，DSA 做稀疏筛选和注意力计算，最后通过 MLA 的输出投影得到结果。

In [5]:
class DSA_MLA(nn.Module):
    def __init__(self, d_model=512, num_query_heads=8, d_k=64, top_k=2048):
        super().__init__()
        self.mla_mqa = MLA_MQA(d_model=d_model, num_query_heads=num_query_heads, d_k=d_k)
        self.dsa = DSA(d_model=d_model, d_k=d_k, top_k=top_k)
        self.W_o = nn.Linear(num_query_heads * d_k, d_model)  # 最终输出投影（和 MLA 复用也可）

    def forward(self, x):
        # x: [batch_size, seq_len, d_model] 输入序列
        # 1. MLQ 生成共享 KV 和多查询头Q
        Q, K, V = self.mla_mqa(x)
        # 2. DSA 稀疏计算
        dsa_output = self.dsa(x, Q, K, V)
        # 3. 输出投影
        output = self.W_o(dsa_output)
        return output

In [6]:
d_model = 512
num_query_heads = 8
d_k = 64
top_k = 10
batch_size = 2
seq_len = 100

x = torch.randn(batch_size, seq_len, d_model)

model = DSA_MLA(d_model=d_model, num_query_heads=num_query_heads, d_k=d_k, top_k=top_k)

output = model(x)

# 验证输出维度（应和输入维度一致）
print(f"输入维度: {x.shape}")  # torch.Size([2, 100, 512])
print(f"输出维度: {output.shape}")  # torch.Size([2, 100, 512])
print("模型运行成功！")

输入维度: torch.Size([2, 100, 512])
输出维度: torch.Size([2, 100, 512])
模型运行成功！
